In [0]:

# ------------------------------------
# Cell 1 — Configuration and Imports
# ------------------------------------

from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.window import Window

# Create Spark session (Databricks usually provides this by default)
spark = SparkSession.builder.appName("DataCleaningPipeline").getOrCreate()

# Define source Delta table names (update as per your workspace)
train_table = "kusha_solutions.telecom_churn_ml.telecom_train"
test_table  = "kusha_solutions.telecom_churn_ml.telecom_test"

print("✅ Configuration and Imports Loaded Successfully")
print(f"Train Table: {train_table}")
print(f"Test Table: {test_table}")



# ------------------------------------
# Cell 2 — Utility Function: Schema + Null Check
# ------------------------------------

from pyspark.sql import functions as F

def show_schema_and_nulls(df, name):
    """
    Displays schema, counts null or empty values per column, 
    and shows a preview of the data.
    """
    print(f"--- Schema: {name} ---")
    df.printSchema()

    exprs = []
    for c, dtype in df.dtypes:
        if dtype in ("string",):
            # Handle missing string values and placeholders
            expr = F.count(
                F.when(
                    (F.col(c).isNull()) | 
                    (F.col(c) == "") | 
                    (F.col(c).isin(["NULL", "None"])), c
                )
            ).alias(c)
        else:
            # Handle missing numeric values
            expr = F.count(F.when(F.col(c).isNull(), c)).alias(c)
        exprs.append(expr)

    null_counts = df.select(exprs)
    null_counts.show(truncate=False)
    print(f"Row count: {df.count()}")
    display(df.limit(5))
    print("✅ Schema and Null Summary Completed")

print("✅ Utility function defined successfully.")





# ------------------------------------
# Cell 3 — Read Train & Test Data from Delta Tables
# ------------------------------------

# Read Delta tables
train_df = spark.table(train_table)
test_df  = spark.table(test_table)

print("✅ Train & Test Data loaded successfully.")

# Check basic info using the utility from Cell 2
show_schema_and_nulls(train_df, "Train Data (Raw)")
show_schema_and_nulls(test_df,  "Test Data (Raw)")

#------------------------------------
# Cell 5 — Data Standardization (Trim, Case, Cast)
# ------------------------------------

from pyspark.sql import functions as F

def standardize_formats(df):
    """
    Cleans up inconsistent formatting and ensures correct data types.
    1. Trims leading/trailing spaces.
    2. Replaces blanks, 'NULL', 'None' with null.
    3. Casts numeric-like columns to correct data types.
    """
    # Identify string columns
    str_cols = [c for c, t in df.dtypes if t == "string"]

    # Trim and replace empty/invalid strings with NULL
    for c in str_cols:
        df = df.withColumn(c, F.trim(F.col(c)))
        df = df.withColumn(
            c,
            F.when(
                (F.col(c) == "") | 
                (F.col(c).isin(["NULL", "None"])), 
                None
            ).otherwise(F.col(c))
        )

    # ✅ Cast numeric columns correctly
    df = df.withColumn("tenure", F.col("tenure").cast("int"))
    df = df.withColumn("SeniorCitizen", F.col("SeniorCitizen").cast("int"))
    df = df.withColumn("MonthlyCharges", F.col("MonthlyCharges").cast("double"))
    df = df.withColumn("TotalCharges", F.col("TotalCharges").cast("double"))

    return df

# Apply to both train and test datasets
train_df = standardize_formats(train_df)
test_df  = standardize_formats(test_df)

print("✅ Data Standardization Completed for Train & Test")
show_schema_and_nulls(train_df, "Train — After Standardization")
show_schema_and_nulls(test_df,  "Test — After Standardization")


#to read and debug
train_df.select("SeniorCitizen").distinct().show()



# ------------------------------------
# Cell 4 — Column Statistics (Descriptive Summary)
# ------------------------------------

from pyspark.sql import functions as F

# 1️⃣ Summary statistics for numeric columns
numeric_cols = [c for c, t in train_df.dtypes if t in ("int", "double", "float", "bigint") and c != 'SeniorCitizen']
print("📊 Numeric Column Summary:")
display(train_df.select(numeric_cols).summary())

# 2️⃣ Distinct counts for categorical columns (to understand diversity of values)
cat_cols = [c for c, t in train_df.dtypes if t == "string" or c=='SeniorCitizen']
print("🔤 Distinct Value Counts for Categorical Columns:")
for c in cat_cols:
    print(f"{c}: {train_df.select(c).distinct().count()} unique values")

# 3️⃣ Optional — show frequency of few important categorical columns
important_cats = ["gender", "Partner", "Dependents", "InternetService", "Contract", "Churn"]
for c in important_cats:
    print(f"\n Top categories in {c}:")
    display(train_df.groupBy(c).count().orderBy(F.desc("count")))





# ------------------------------------
# Cell 6 — Handling Missing Values (Null Imputation)
# ------------------------------------

from pyspark.sql import functions as F

# 1️⃣ Show initial missing value summary
print("🔍 Checking missing values before imputation:")
show_schema_and_nulls(train_df, "Train Data Before Imputation")

# 2️⃣ Fill missing numeric columns with median
numeric_cols = [c for c, t in train_df.dtypes if t in ('int', 'double', 'float','bigint') and c != 'SeniorCitizen']
for col_name in numeric_cols:
    median_val = train_df.approxQuantile(col_name, [0.5], 0.01)[0]
    train_df = train_df.fillna({col_name: median_val})

# 3️⃣ Fill missing categorical columns with mode (most frequent value)
categorical_cols = [c for c, t in train_df.dtypes if t == 'string' or c == 'SeniorCitizen']
for col_name in categorical_cols:
    mode_val = (
        train_df.groupBy(col_name)
        .count()
        .orderBy(F.desc("count"))
        .first()
    )
    if mode_val:
        train_df = train_df.fillna({col_name: mode_val[0]})

# 4️⃣ Validate after imputation
print("\n✅ Missing value imputation complete. Verifying...")
show_schema_and_nulls(train_df, "Train Data After Imputation")

print("✅ Null handling completed successfully.")


#to read and debug
print("debugging************************")
train_df.select("SeniorCitizen").distinct().show()



# ------------------------------------
# Cell 8 — Outlier Detection and Capping (IQR Method)
# ------------------------------------

from pyspark.sql import functions as F

# 1️⃣ Identify numeric columns for outlier treatment
numeric_cols = [c for c, t in train_df.dtypes if t in ('int', 'double', 'float','bigint') and c != 'SeniorCitizen']

# 2️⃣ Compute IQR (Interquartile Range) for each numeric column
iqr_bounds = {}

for col_name in numeric_cols:
    try:
        q1, q3 = train_df.approxQuantile(col_name, [0.25, 0.75], 0.01)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        iqr_bounds[col_name] = (lower_bound, upper_bound)
    except Exception as e:
        print(f"⚠️ Skipping {col_name}: {e}")

# 3️⃣ Apply capping for outliers
for col_name, (lower, upper) in iqr_bounds.items():
    train_df = train_df.withColumn(
        col_name,
        F.when(F.col(col_name) < lower, lower)
         .when(F.col(col_name) > upper, upper)
         .otherwise(F.col(col_name))
    )

print("✅ Outlier detection and capping completed using IQR method.")

# 4️⃣ Optional: Show example column statistics after capping
for col_name in numeric_cols[:5]:  # show first 5 columns only
    stats = train_df.select(
        F.min(col_name).alias("min"),
        F.expr(f'percentile({col_name}, 0.5)').alias("median"),
        F.max(col_name).alias("max")
    ).first()
    print(f"{col_name}: min={stats['min']}, median={stats['median']}, max={stats['max']}")



#to read and debug
print("debugging************************")
train_df.select("SeniorCitizen").distinct().show()


# ------------------------------------
# Cell 9 — Apply Cleaning (Test Data)
# ------------------------------------

from pyspark.sql import functions as F

# 1️⃣ Show missing values before imputation
print("🔍 Checking missing values in Test Data before imputation:")
show_schema_and_nulls(test_df, "Test Data Before Imputation")

# 2️⃣ Fill missing numeric columns in test_df with median values from train_df
numeric_cols = [c for c, t in test_df.dtypes if t in ('int', 'double', 'float') and c != 'SeniorCitizen']
for col_name in numeric_cols:
    median_val = train_df.approxQuantile(col_name, [0.5], 0.01)[0]
    test_df = test_df.fillna({col_name: median_val})

# 3️⃣ Fill missing categorical columns with mode values from train_df
categorical_cols = [c for c, t in test_df.dtypes if t == 'string' or c == 'SeniorCitizen']
for col_name in categorical_cols:
    mode_val = (
        train_df.groupBy(col_name)
        .count()
        .orderBy(F.desc("count"))
        .first()
    )
    if mode_val:
        test_df = test_df.fillna({col_name: mode_val[0]})

# 4️⃣ Handle outliers in numeric columns (using train_df fences)
for col_name in numeric_cols:
    Q1, Q3 = train_df.approxQuantile(col_name, [0.25, 0.75], 0.01)
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR
    test_df = test_df.withColumn(
        col_name,
        F.when(F.col(col_name) < lower_fence, lower_fence)
         .when(F.col(col_name) > upper_fence, upper_fence)
         .otherwise(F.col(col_name))
    )

# 5️⃣ Validate final test_df
print("\n✅ Test Data cleaning complete. Verifying...")
show_schema_and_nulls(test_df, "Test Data After Cleaning")

print("✅ Test dataset cleaned successfully and ready for transformation.")


#to read and debug
print("debugging************************")
train_df.select("SeniorCitizen").distinct().show()


# ------------------------------------
# Cell 10 — Save Cleaned Data as Delta Tables
# ------------------------------------
 
clean_train_table = "kusha_solutions.telecom_churn_ml.telecom_train_clean"
clean_test_table  = "kusha_solutions.telecom_churn_ml.telecom_test_clean"
 
# Overwrite mode ensures updated data replaces previous versions
train_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(clean_train_table)
test_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(clean_test_table)
 
print(f"✅ Cleaned Train Data saved as: {clean_train_table}")
print(f"✅ Cleaned Test Data saved as: {clean_test_table}")


#to read and debug
print("debugging************************")
train_df.select("SeniorCitizen").distinct().show()




In [0]:
from pyspark.sql import functions as F

# 1️⃣ Summary statistics for numeric columns
numeric_cols = [c for c, t in train_df.dtypes if t in ("int", "double", "float", "bigint")]
print("📊 Numeric Column Summary:")
display(train_df.select(numeric_cols).summary())

# 2️⃣ Distinct counts for categorical columns (to understand diversity of values)
cat_cols = [c for c, t in train_df.dtypes if t == "string"]
print("🔤 Distinct Value Counts for Categorical Columns:")
for c in cat_cols:
    print(f"{c}: {train_df.select(c).distinct().count()} unique values")

# 3️⃣ Optional — show frequency of few important categorical columns
important_cats = ["gender", "Partner", "Dependents", "InternetService", "Contract", "Churn"]
for c in important_cats:
    print(f"\n Top categories in {c}:")
    display(train_df.groupBy(c).count().orderBy(F.desc("count")))